Load dataframes with acf and calculate tau

In [3]:
import pandas as pd
import numpy as np

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_tau import fit_single_exp
from cfg_global import project_folder_path
os.chdir(current_wd)

### Get the data

In [4]:
results_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms\\'

In [5]:
area = 'pfdl' # pfp

In [6]:
n_lags = 20
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


In [10]:
# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau(pearsonr_trial_avg_df, acf_cols)
# pfdl_tau_pearsonr_df.to_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
# pfdl_tau_pearsonr_df.query('tau_ms > 0 and tau_ms <= 1000').describe()

Calculating taus for (544, 20)
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf 

In [8]:
pearsonr_trial_avg_df

,unit_id,acf_0,acf_1,acf_2,acf_3,acf_4,acf_5,acf_6,acf_7,acf_8,...,acf_10,acf_11,acf_12,acf_13,acf_14,acf_15,acf_16,acf_17,acf_18,acf_19
0,0,1.0,0.444373,0.430208,0.376476,0.334626,0.286711,0.241219,0.189917,0.195370,...,0.192167,0.149290,0.217536,0.150978,0.158743,0.112176,0.021287,0.044947,0.058803,-0.056918
1,1,1.0,0.039497,0.100905,0.026464,0.060659,0.084223,0.060774,0.149708,-0.025173,...,-0.024061,0.089305,-0.021454,0.147307,-0.021161,-0.020168,-0.017662,-0.017437,-0.019246,NaN
2,2,1.0,0.088925,0.064711,0.060999,0.022718,0.070518,0.094123,0.033764,0.034610,...,0.013201,0.037633,0.052727,0.027214,0.032464,0.247909,0.091537,-0.053734,0.122978,-0.071310
3,3,1.0,0.439559,0.394372,0.374076,0.315065,0.297622,0.315547,0.285790,0.263921,...,0.270382,0.247299,0.278330,0.221721,0.237938,0.211978,0.156498,0.198861,0.231307,0.226090
4,4,1.0,0.139126,0.160558,0.081982,0.062575,0.041640,0.025653,0.046473,0.040326,...,0.017495,0.032412,0.010911,0.047202,0.000081,-0.004478,0.028326,-0.053410,0.052446,0.042030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,539,1.0,0.259138,0.224603,0.221199,0.133688,0.116029,0.137831,0.165647,0.123091,...,0.126383,0.160318,0.120592,0.143463,0.079920,0.207331,0.058086,0.079398,0.143355,-0.054183
540,540,1.0,0.229299,0.257028,0.092984,0.027435,0.041177,-0.032945,-0.031200,-0.031823,...,-0.032228,0.005750,-0.023680,-0.022400,-0.023159,-0.018562,-0.015923,-0.020114,-0.020068,-0.019141
541,541,1.0,0.458175,0.338126,0.220629,0.139172,0.044472,-0.027294,-0.055418,-0.055126,...,-0.013676,-0.002172,0.007689,0.000764,0.010235,0.028578,-0.013256,0.022176,-0.026484,-0.034636
542,542,1.0,0.157685,0.166556,0.117979,0.085931,0.000590,0.024190,-0.038476,0.011745,...,-0.033436,0.071445,0.008283,0.075503,-0.030348,0.002666,-0.007712,-0.000141,-0.062219,-0.048498


In [11]:
tau_pearsonr_trial_avg_df

,unit_id,fit_a,fit_b,fit_c,tau,r_squared,tau_ms
0,0,0.812323,0.040859,-0.358625,24.474453,0.891188,1.223723e+03
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.564505,1.868106,0.051750,0.535302,0.002196,2.676508e+01
3,3,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.453725,0.617403,0.022986,1.619689,0.627602,8.098445e+01
...,...,...,...,...,...,...,...
539,539,151.296790,0.000048,-151.093494,20843.040237,0.358880,1.042152e+06
540,540,1.421809,0.817298,-0.022154,1.223544,0.958271,6.117722e+01
541,541,1.232861,0.597492,-0.015048,1.673662,0.906138,8.368310e+01
542,542,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau(sttc_trial_avg_df, acf_cols)

Calculating taus for (544, 20)


In [13]:
tau_sttc_trial_avg_df

,unit_id,fit_a,fit_b,fit_c,tau,r_squared,tau_ms
0,0,0.351602,0.071483,0.160833,13.989436,0.913324,699.471824
1,1,11.626593,0.000524,-11.521032,1908.836333,0.450336,95441.816655
2,2,4850.978001,7.445680,0.074394,0.134306,0.000028,6.715303
3,3,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.387578,0.631361,0.084819,1.583879,0.501735,79.193958
...,...,...,...,...,...,...,...
539,539,NaN,NaN,NaN,NaN,NaN,NaN
540,540,0.739815,0.656450,-0.012689,1.523345,0.949186,76.167235
541,541,0.807141,0.556572,0.138495,1.796714,0.917859,89.835702
542,542,NaN,NaN,NaN,NaN,NaN,NaN


### Calculate tau

In [9]:
def calc_tau(acf_df_, acf_cols_):
    acf_2d = acf_df_[acf_cols_].values
    acf_2d = acf_2d.astype(np.float64) # RuntimeWarning: overflow encountered in matmul sttc start_idx 2
    n_units = acf_2d.shape[0]
    print('Calculating taus for {}'.format(acf_2d.shape))
    
    fit_popt_a_l, fit_popt_b_l, fit_popt_c_l = [],[],[]
    fit_tau_l = []
    fit_r_squared_l = []
    
    for i in range(n_units):
        fit_popt, fit_pcov, tau, fit_r_squared = fit_single_exp(acf_2d[i,:], start_idx_=2)
        if  type(fit_popt) == np.ndarray:
            fit_popt_a_l.append(fit_popt[0])
            fit_popt_b_l.append(fit_popt[1])
            fit_popt_c_l.append(fit_popt[2])
        else:
            fit_popt_a_l.append(np.nan)
            fit_popt_b_l.append(np.nan)
            fit_popt_c_l.append(np.nan)
        fit_tau_l.append(tau)
        fit_r_squared_l.append(fit_r_squared)
    
    data_df = np.vstack((fit_popt_a_l, fit_popt_b_l, fit_popt_c_l, fit_tau_l, fit_r_squared_l)).T
    tau_df = pd.DataFrame(data_df, columns=['fit_a', 'fit_b', 'fit_c','tau', 'r_squared'])
    tau_df.insert(0, 'unit_id', acf_df_['unit_id'].values)
    # tau_df.insert(1, 'trial_id', acf_proper_df['trial_id'].values)
    tau_df['tau_ms'] = tau_df['tau'] * 50

    return tau_df

In [ ]:
# # pfdl pearsonr
# pfdl_acf_pearsonr_df = pd.read_pickle(results_folder + '\\binned\\pfdl\\acf\\acf_average_trial_pearsonr_with_empty_50ms_20lags_df.pkl')
# pfdl_tau_pearsonr_df = calc_tau(pfdl_acf_pearsonr_df, acf_cols)
# pfdl_tau_pearsonr_df.to_pickle(save_folder + '\\binned\\pfdl\\taus\\tau_acf_average_trial_pearsonr_with_empty_50ms_20lags_ind2_df.pkl')
# pfdl_tau_pearsonr_df.query('tau_ms > 0 and tau_ms <= 1000').describe()

# # pfp pearsonr
# pfp_acf_pearsonr_df = pd.read_pickle(results_folder + '\\binned\\pfp\\acf\\acf_average_trial_pearsonr_with_empty_50ms_20lags_df.pkl')
# pfp_tau_pearsonr_df = calc_tau(pfp_acf_pearsonr_df, acf_cols)
# pfp_tau_pearsonr_df.to_pickle(save_folder + '\\binned\\pfp\\taus\\tau_acf_average_trial_pearsonr_with_empty_50ms_20lags_ind2_df.pkl')
# pfp_tau_pearsonr_df.query('tau_ms > 0 and tau_ms <= 1000').describe()

# # pfdl sttc
# pfdl_acf_sttc_df = pd.read_pickle(results_folder + '\\non_binned\\pfdl\\acf\\acf_average_trial_sttc_with_empty_50ms_20lags_df.pkl')
# pfdl_tau_sttc_df = calc_tau(pfdl_acf_sttc_df, acf_cols)
# pfdl_tau_sttc_df.to_pickle(save_folder + '\\non_binned\\pfdl\\taus\\tau_acf_average_trial_sttc_with_empty_50ms_20lags_ind2_df.pkl')
# pfdl_tau_sttc_df.query('tau_ms > 0 and tau_ms <= 1000').describe()

# pfp sttc
pfp_acf_sttc_df = pd.read_pickle(results_folder + '\\non_binned\\pfp\\acf\\acf_average_trial_sttc_with_empty_50ms_20lags_df.pkl')
pfp_tau_sttc_df = calc_tau(pfp_acf_sttc_df, acf_cols)
pfp_tau_sttc_df.to_pickle(save_folder + '\\non_binned\\pfp\\taus\\tau_acf_average_trial_sttc_with_empty_50ms_20lags_ind2_df.pkl')
pfp_tau_sttc_df.query('tau_ms > 0 and tau_ms <= 1000').describe()